## TC 5033
### Word Embeddings

<br>

#### Activity 3b: Text Classification using RNNs and AG_NEWS dataset in PyTorch
<br>

- Objective:
    - Understand the basics of Recurrent Neural Networks (RNNs) and their application in text classification.
    - Learn how to handle a real-world text dataset, AG_NEWS, in PyTorch.
    - Gain hands-on experience in defining, training, and evaluating a text classification model in PyTorch.
    
<br>

- Instructions:
    - Data Preparation: Starter code will be provided that loads the AG_NEWS dataset and prepares it for training. Do not modify this part. However, you should be sure to understand it, and comment it, the use of markdown cells is suggested. 

    - Model Setup: A skeleton code for the RNN model class will be provided. Complete this class and use it to instantiate your model.

    - Implementing Accuracy Function: Write a function that takes model predictions and ground truth labels as input and returns the model's accuracy.

    - Training Function: Implement a function that performs training on the given model using the AG_NEWS dataset. Your model should achieve an accuracy of at least 80% to get full marks for this part.

    - Text Sampling: Write a function that takes a sample text as input and classifies it using your trained model.

    - Confusion Matrix: Implement a function to display the confusion matrix for your model on the test data.

    - Submission: Submit your completed Jupyter Notebook. Make sure to include a markdown cell at the beginning of the notebook that lists the names of all team members. Teams should consist of 3 to 4 members.
    
<br>

- Evaluation Criteria:

    - Correct setup of all the required libraries and modules (10%)
    - Code Quality (30%): Your code should be well-organized, clearly commented, and easy to follow. Use also markdown cells for clarity. Comments should be given for all the provided code, this will help you understand its functionality.
    
   - Functionality (60%): 
        - All the functions should execute without errors and provide the expected outputs.
        - RNN model class (20%)
        - Accuracy fucntion (10%)
        - Training function (10%)
        - Sampling function (10%)
        - Confucion matrix (10%)

        - The model should achieve at least an 80% accuracy on the AG_NEWS test set for full marks in this criterion.


Dataset

https://pytorch.org/text/stable/datasets.html#text-classification

https://paperswithcode.com/dataset/ag-news


### Import libraries

In [ ]:
# conda install -c pytorch torchtext
# conda install -c pytorch torchdata
# conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

In [1]:
# The following libraries are required for running the given code
# Please feel free to add any libraries you consider adecuate to complete the assingment.
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
# Dataloader library
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F

# These libraries are suggested to plot confusion matrix
# you may use others
import scikitplot as skplt
import gc

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    # If using Mac M1/M2
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(device)

mps


### Get the train and the test datasets and dataloaders

Classes:

* 1 - World

* 2 - Sports

* 3 - Business

* 4 - Sci/Tech

We will convert them to:

* 0 - World

* 1 - Sports

* 2 - Business

* 3 - Sci/Tech

In [3]:
train_dataset, test_dataset = AG_NEWS()
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [4]:
# Get the tokeniser
# tokeniser object
tokeniser = get_tokenizer('basic_english')

def yield_tokens(data):
    for _, text in data:
        yield tokeniser(text)

In [5]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [6]:
#test tokens
tokens = tokeniser('Welcome to TE3007')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007'] [3314, 4, 0]


In [7]:
NUM_TRAIN = int(len(train_dataset)*0.9)
NUM_VAL = len(train_dataset) - NUM_TRAIN

In [8]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [9]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

108000 12000 7600


In [10]:
# function passed to the DataLoader to process a batch of data as indicated
def collate_batch(batch):
    # Get label and text
    y, x = list(zip(*batch))
    
    # Create list with indices from tokeniser
    x = [vocab(tokeniser(text)) for text in x]
    x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]

    # Prepare the labels, by subtracting 1 to get them in the range 0-3
    return torch.tensor(x, dtype=torch.int32), torch.tensor(y, dtype=torch.int32) - 1

In [11]:
labels =  ["World", "Sports", "Business", "Sci/Tech"]
max_tokens = 50
BATCH_SIZE = 256

In [12]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)

### Let us build our RNN model

In [19]:
train_loader.dataset[0]

(4,
 'PalmOS Goes Linux PalmSource, the makers of the PalmOS are looking to acquire Chinas MobileSoft. MobileSoft will provide PalmSource with a solid footing in the lucrative Chinese and Asian markets, as well as providing valuable ')

In [65]:
for i in range(10):
    print(len(train_loader.dataset[0][1].split(' ')))

36
36
36
36
36
36
36
36
36
36


In [44]:
EMBEDDING_SIZE = 10 # complete
NEURONS = 10# complete
LAYERS = 10 # complete
NUM_CLASSES = 4 # complete

In [148]:
class RnnModel(nn.Module):
    def __init__(self, embed_size, hidden, layers, num_classes):
        super().__init__()
        
        self.embed_size = embed_size
        self.layers = layers
        self.hidden = hidden
        self.num_classes = num_classes
        
        self.embedding_layer = nn.Embedding(
            num_embeddings=len(vocab), 
            embedding_dim=embed_size
        )
        
        self.rnn = nn.RNN(10, hidden, layers, batch_first=True) 
        # complete this code/
        # You may use PyTorch nn.GRU(), nn.RNN(), or nn.LSTM()
        
        self.fc = nn.Linear(hidden, num_classes) 
        # complete output classifier layer using linear layer
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding_layer(x)
        hidden = torch.zeros(self.layers, batch_size, self.hidden)
        hidden = hidden.to(device=device, dtype=torch.float)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:,-1,:])
        return out
        # implement forward pass. This function will be called when executing the model

In [190]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    cost = 0.
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for mb, (xi, yi) in enumerate(loader):
            xi = xi.to(device=device, dtype = torch.long)
            yi = yi.to(device=device, dtype = torch.long)
            scores = model(xi)
            cost += (F.cross_entropy(scores, yi)).item()
            _, pred = scores.max(dim=1)
            num_correct += (pred == yi.squeeze()).sum()
            num_total += pred.size(0)

    return cost/mb, float(num_correct)/num_total

In [174]:
import time
import datetime

t1 = time.time()

time.sleep(2)

t2 = time.time()

print(t2 - t1)
print(type(t2-t1))


2.0069470405578613
<class 'float'>


In [191]:
def train(model, optimiser, epochs=100):
    model = model.to(device=device)
    train_cost = 0.
    val_cost = 0.
    
    print("## Total number of batches: {}".format(len(train_loader)))
    
    last_batch_time = 1
    remaining_batches = -1
    remaining_epochs = epochs
    batches = len(train_loader)
    
    for epoch in range(epochs):
        train_correct_num  = 0.
        train_total = 0.
        train_cost_acum = 0
        remaining_batches = batches
        for mb, (xi, yi) in enumerate(train_loader):
            print("\r## Epoch {}/{} ..... Batch {}/{} ..... Epoch ETA {} ..... Training ETA {}".format(
                epoch + 1, 
                epochs, 
                mb + 1, 
                batches,
                datetime.timedelta(
                    seconds=int(remaining_batches * last_batch_time)
                ),
                datetime.timedelta(
                    seconds=int(
                        (remaining_batches * last_batch_time) + ((remaining_epochs - 1) * batches * last_batch_time)
                    )
                )
            ), end="")
            t1 = time.time()
            model.train()
            xi = xi.to(device=device, dtype=torch.long)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)
            cost = F.cross_entropy(input= scores, target=yi.squeeze())
            optimiser.zero_grad()
            cost.backward()
            optimiser.step()

            train_correct_num += (torch.argmax(scores, dim=1) == yi.squeeze()).sum()
            train_total += scores.size(0)

            train_cost_acum += cost.item() 
            t2 = time.time()
            last_batch_time = t2 - t1
            remaining_batches -= 1

        val_cost, val_acc = accuracy(model, val_loader)
        train_acc = float(train_correct_num)/train_total
        train_cost = train_cost_acum/mb
        remaining_epochs -= 1
        
        print("\r## Epoch {}/{} ... Train Cost {:.6f} ... Val Cost {:.6f} ... Train Acc {:.6f} ... Val Acc {:.6f}".format(
            epoch + 1, epochs,
            train_cost, val_cost,
            train_acc, val_acc
        ))

        #print(f'Epoch:{epoch}, train cost: {train_cost:.6f}, val cost: {val_cost:.6f},'
        #          f' train acc: {train_acc:.4f}, val acc: {val_acc:4f},'
        #          f' lr: {optimiser.param_groups[0]["lr"]:.6f}')

In [192]:
epochs = 10 # define
lr = 0.01 # to do
# instantiate model
rnn_model = RnnModel(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)
optimiser = torch.optim.Adam(rnn_model.parameters(), lr=lr)


In [193]:
train(rnn_model, optimiser=optimiser,  epochs=epochs)

## Total number of batches: 422
## Epoch 1/10 ... Train Cost 1.391132 ... Val Cost 1.416355 ... Train Acc 0.250685 ... Val Acc 0.258583
## Epoch 2/10 ... Train Cost 1.390176 ... Val Cost 1.416866 ... Train Acc 0.249944 ... Val Acc 0.244833
## Epoch 3/10 ... Train Cost 1.390463 ... Val Cost 1.416354 ... Train Acc 0.251926 ... Val Acc 0.258583
## Epoch 4/10 ... Train Cost 1.389895 ... Val Cost 1.417962 ... Train Acc 0.248435 ... Val Acc 0.244833
## Epoch 5/10 ... Train Cost 1.390352 ... Val Cost 1.416656 ... Train Acc 0.252231 ... Val Acc 0.245500
## Epoch 6/10 ... Train Cost 1.389971 ... Val Cost 1.416232 ... Train Acc 0.250250 ... Val Acc 0.258583
## Epoch 7/10 ... Train Cost 1.389941 ... Val Cost 1.416312 ... Train Acc 0.249361 ... Val Acc 0.258583
## Epoch 8/10 ... Train Cost 1.389937 ... Val Cost 1.416463 ... Train Acc 0.249593 ... Val Acc 0.251083
## Epoch 9/10 ... Train Cost 1.392536 ... Val Cost 1.429499 ... Train Acc 0.249593 ... Val Acc 0.251083
## Epoch 10/10 ... Train Cost 1.

In [31]:
print(f'{accuracy(rnn_model, test_loader):.4f}')

tensor([[8.5100e+02, 3.4600e+02, 3.5100e+02,  ..., 1.6700e+02, 1.0000e+00,
         4.4000e+01],
        [1.1830e+03, 3.6400e+03, 6.6000e+01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0100e+02, 4.0000e+00, 6.1200e+02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [1.5890e+03, 1.2900e+02, 6.3010e+03,  ..., 2.8810e+03, 1.0000e+00,
         0.0000e+00],
        [3.3900e+02, 5.6850e+03, 3.7200e+02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.2800e+02, 9.4300e+02, 6.6000e+01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='mps:0')


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got MPSFloatType instead (while checking arguments for embedding)

In [32]:
def sample_text(model, loader):
    pass

In [33]:
sample_text(rnn_model, test_loader)

In [ ]:
# create confusion matrix
pass